# Imports

In [1]:
import re
import string

import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 150)

from preprocessing import tokenize

In [2]:
DATA_PATH_PREP = '../DATA/prepared'

# Load data

In [3]:
df_full = pd.read_pickle(f'{DATA_PATH_PREP}/02_df_full_text_no_meta.pkl')
df_full.head()

,author,title,download_link,local_filename,text
0,ivan_vazov,Чичовци,https://chitanka.info/text/3757-chichovtsi.txt.zip,../DATA/original/ivan_vazov_tbc_chichovtsi.txt,\n\tГалерия от типове и нрави български в турско време\n\n\n\tI. Общество\n\n\tУтринното лятно слънце изскокна високо над Стара планина. Потоци от...
1,ivan_vazov,Под игото,https://chitanka.info/text/3753-pod-igoto.txt.zip,../DATA/original/ivan_vazov_tbc_pod_igoto.txt,"\n\n\n\tЧаст първа\n\n\tI. Гост\n\n\tТая прохладна майска вечер чорбаджи Марко, гологлав, по халат, вечеряше с челядта си на двора.\n\tГосподарска..."
2,ivan_vazov,Българският език,https://chitanka.info/text/5189-bylgarskijat-ezik.txt.zip,../DATA/original/ivan_vazov_bylgarskijat_ezik.txt,"\n\n\n\n\tЕзик свещен на моите деди\n\tезик на мъки, стонове вековни,\n\tезик на тая, дето ни роди\n\tза радост не — за ядове отровни.\n\n\tЕзик п..."
3,ivan_vazov,Кочо,https://chitanka.info/text/3851-kocho.txt.zip,../DATA/original/ivan_vazov_kocho.txt,"\n\tЗащитата на Перущица\n\n\n\tO, движенье славно, о, мрачно движенье,\n\tдни на борба горда, о, дни на паденье!\n\tЕпопея тъмна, непозната нам,\..."
4,ivan_vazov,Левски,https://chitanka.info/text/3849-levski.txt.zip,../DATA/original/ivan_vazov_levski.txt,"\n\n\n\tМанастирът тесен за мойта душа е.\n\tКога човек дойде тук да се покае,\n\tтрябва да забрави греховния мир,\n\tда бяга съблазни и да търси ..."


# Create samples

In [4]:
df_group_by_author = df_full.groupby("author")["text"].agg(" ".join).reset_index()
df_group_by_author

,author,text
0,aleko-konstantinov,"\n\n\n\tНашият помощник на регистратора е страшен комик. Изпокапваме от смях, когато почне да бомбардира несправедливата съдба.\n\t— Е, дявол да г..."
1,dimityr-dimov,"\n\n\n\tI глава\n\n\tГроздоберът беше към края си.\n\tОт малките вили и кирпичените постройки сред лозята се разнасяха ту задружни песни, бързи и ..."
2,dimityr-talev,"\n\n\n\tПърва част\n\tХаджи Серафимовата внука\n\n\n\n\tОвде дърво столовито,\n\tстоловито, грановито,\n\tгранки му са до небеси,\n\tа корени — су..."
3,elin-pelin,"\n\n\n\tПърва глава\n\n\tНай-заможният човек в селото беше дядо Йордан Геракът. Пъргав и трудолюбив, той бе работил през целия си живот и бе сполу..."
4,ivan_vazov,\n\tГалерия от типове и нрави български в турско време\n\n\n\tI. Общество\n\n\tУтринното лятно слънце изскокна високо над Стара планина. Потоци от...
5,jordan-jovkov,"\n\n\n\tI\n\n\tГрадът бързо се подновяваше и растеше. Но все пак оставаха много улици, където още можеше да се видят тихи, едновремешни къщурки, с..."


In [5]:
limit_x = df_group_by_author['text'].map(lambda t: len(t.strip().split())).min() - 1000
print(f'{limit_x=}')
limit_y = 1000

num_authors = df_group_by_author.shape[0]
num_samples = 100

start_idxs = np.random.randint(0, limit_x, (num_authors, num_samples))
sample_lengths = np.random.randint(3, limit_y, (num_authors, num_samples))

df_samples = pd.DataFrame(columns=['text', 'author'])
for i in range(num_authors):
    author_name = df_group_by_author['author'][i]
    for j, start_idx in enumerate(start_idxs[i]):
        sample = df_group_by_author['text'][i][start_idx : (start_idx + sample_lengths[i, j])]
        df_samples.loc[len(df_samples)] = [sample, author_name]
df_samples

limit_x=29374


,text,author
0,"сията й — злато с шепа, моля ти се. Късмет ли да го наречеш, какво да го наречеш — не знайш! Па вземи и децата й: по-голямото, не го ли знаете, ед...",aleko-konstantinov
1,сега не си раз,aleko-konstantinov
2,"ме какво да правя.\n\t— Как какво да правиш! Прави каквото правят хората: свий си опашката, па си налягай парцалите… Какво се кикотиш, не е ли тъй...",aleko-konstantinov
3,"а, че не съчувствувай на македонците…\n\t(Я тури две-три дървета в печката. Стига.)\n\tНямаше си хас, джанъм, този Биконсфилд, дето ни разпокъса. ...",aleko-konstantinov
4,"шва)… значи, и да се разгатне енигмата на зеленото сукно с металическото копче, и пламъкът на петрола в нощната тъмнина, и ужасният смрад, и цвърт...",aleko-konstantinov
...,...,...
595,"я занятието си с друго, не тъй почтено и красиво: продаваше семки, фъстъци и лешници. Малко усърдие влагаше той в тая работа и всеки път свършваше...",jordan-jovkov
596,"с една галантност, в която се съглежда не умение, а продължителна дресировка, кланя се и я поднася на Цветана.\n\t— Зюмбюли — напевно говори той....",jordan-jovkov
597,"то градските учители и учителки напрягаха всичките си сили и мислеха, че достигат вече Коклена и Сара Бернар, изведнъж вратата на салона се отваря...",jordan-jovkov
598,"азник на панаира, за разходките по шосето, за много други неща още. Но веднага, със светнали от вълнение очи, те ще се запитат: „А помниш ли дядо ...",jordan-jovkov


# Tokenize

In [6]:
test = 'Това е низ! Съдържа препинателни "знаци" и цифри като: 42, 420, 4200 и 7.\nКак можем да го разделим??'

print('################ Soft ################')
print(tokenize(test, 'soft'))
print()

print('################ Medium ################')
print(tokenize(test, 'medium'))
print()

print('################ Hard ################')
print(tokenize(test, 'hard'))

################ Soft ################
['това', 'е', 'низ', '!', 'съдържа', 'препинателни', '"', 'знаци', '"', 'и', 'цифри', 'като', ':', '42', ',', '420', ',', '4200', 'и', '7', '.', 'как', 'можем', 'да', 'го', 'разделим', '?', '?', 'това е', 'е низ', 'низ !', '! съдържа', 'съдържа препинателни', 'препинателни "', '" знаци', 'знаци "', '" и', 'и цифри', 'цифри като', 'като :', ': 42', '42 ,', ', 420', '420 ,', ', 4200', '4200 и', 'и 7', '7 .', '. как', 'как можем', 'можем да', 'да го', 'го разделим', 'разделим ?', '? ?', 'това е низ', 'е низ !', 'низ ! съдържа', '! съдържа препинателни', 'съдържа препинателни "', 'препинателни " знаци', '" знаци "', 'знаци " и', '" и цифри', 'и цифри като', 'цифри като :', 'като : 42', ': 42 ,', '42 , 420', ', 420 ,', '420 , 4200', ', 4200 и', '4200 и 7', 'и 7 .', '7 . как', '. как можем', 'как можем да', 'можем да го', 'да го разделим', 'го разделим ?', 'разделим ? ?', 'това е низ !', 'е низ ! съдържа', 'низ ! съдържа препинателни', '! съдържа препин

In [9]:
df_full_soft = df_samples.copy()
df_full_soft['text'] = df_full_soft['text'].map(lambda t: tokenize(t, 'soft'))

df_full_med = df_samples.copy()
df_full_med['text'] = df_full_med['text'].map(lambda t: tokenize(t, 'medium'))

df_full_hard = df_samples.copy()
df_full_hard['text'] = df_full_hard['text'].map(lambda t: tokenize(t, 'hard'))

In [10]:
print('################ Heads ################')
print(df_full_soft['text'][0][:20])
print(df_full_med['text'][0][:20])
print(df_full_hard['text'][0][:20])

print('################ Tails ################')
print(df_full_soft['text'][0][-20:])
print(df_full_med['text'][0][-20:])
print(df_full_hard['text'][0][-20:])

################ Heads ################
['сията', 'й', '—', 'злато', 'с', 'шепа', ',', 'моля', 'ти', 'се', '.', 'късмет', 'ли', 'да', 'го', 'наречеш', ',', 'какво', 'да', 'го']
['сия', 'неин', '—', 'злато', 'с', 'шепа', ',', 'моля', 'ти', 'се', '.', 'късмет', 'ли', 'да', 'то', 'нарека', ',', 'какъв', 'да', 'то']
['сия', 'злато', 'шепа', 'късмет', 'наре', 'наре', 'знайм', 'па', 'взема', 'дете', 'погол', 'зная', 'голобрад', 'таквоз', 'никак', 'хлапя', 'взема', 'натрап', 'булк', 'разбира']
################ Tails ################
['кой хал съм !', 'хал съм ! и', 'съм ! и аз', '! и аз уж', 'и аз уж се', 'аз уж се увирам', 'уж се увирам около', 'се увирам около големците', 'увирам около големците :', 'около големците : нова', 'големците : нова година', ': нова година било', 'нова година било ,', 'година било , имен', 'било , имен ден', ', имен ден било', 'имен ден било ,', 'ден било , все', 'било , все ,', ', все , з']
['кой хал съм !', 'хал съм ! и', 'съм ! и аз', '! и аз уж', 'и аз уж се',

In [11]:
df_full_hard

,text,author
0,"[сия, злато, шепа, късмет, наре, наре, знайм, па, взема, дете, погол, зная, голобрад, таквоз, никак, хлапя, взема, натрап, булк, разбира, знам, из...",aleko-konstantinov
1,[раз],aleko-konstantinov
2,"[правя, правя, какъвто, правя, хора, свий, опашк, па, наляга, кикот, ами, няма, опашк, смей, смей, дойд, плача, дойд, чува, казва, запомн, хубав, ...",aleko-konstantinov
3,"[съчувствува, туря, дветър, дърв, печк, стига, няма, хас, джанъм, биконсфилд, дето, разпокъса, дип, санстефанск, българ, трябва, цепя, ида, мига, ...",aleko-konstantinov
4,"[знача, разгат, енигм, зелен, сукн, металическ, копче, пламъ, петрол, нощен, тъмни, ужас, смрад, закри, фаетон, драгоманск, дефил, уу, кос, баст, ...",aleko-konstantinov
...,...,...
595,"[заняти, друго, почтен, красив, продава, семк, фъстъ, лешниц, усърди, влага, тая, работ, път, свършва, загуб, чове, меко, нежно, сърц, мога, устоя...",jordan-jovkov
596,"[галантност, съглежда, умени, продължител, дресиров, кланя, поднася, цвета, зюмбюл, напев, говор, радост, сърц, емблематич, значени, веднаг, приба...",jordan-jovkov
597,"[градск, учит, учителк, напряга, всичк, силя, мисл, достига, кокл, сара, бернар, изведнъж, врата, салон, отваря, оглушител, трясъ, влиза, висок, п...",jordan-jovkov
598,"[азни, панаир, разходк, шосе, нещо, веднаг, светн, вълнени, око, запита, помн, дядо, славя, рачо, самсара, люцка, тия, дветър, име, изведнъж, пора...",jordan-jovkov


# Saving to files

In [12]:
df_samples.to_pickle(f'{DATA_PATH_PREP}/03_df_samples.pkl')

In [13]:
df_full_soft.to_pickle(f'{DATA_PATH_PREP}/03_df_full_soft.pkl')
df_full_med.to_pickle(f'{DATA_PATH_PREP}/03_df_full_med.pkl')
df_full_hard.to_pickle(f'{DATA_PATH_PREP}/03_df_full_hard.pkl')